# March Madness Data Analysis Log

In [10]:
import pandas as pd

df = pd.read_csv(
        "https://raw.githubusercontent.com/IGARDS/RPLib/master/data/dataset_tool_datasets.tsv",sep='\t')

df_games = df.loc[(df['Type'] == 'Games') & (df['Dataset Name'].str.contains('March Madness'))]
df_games

,Dataset ID,Dataset Name,Description,Type,Data provenance,Download links
1,1,March Madness 2002,"2002 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...


## D Matrix Creation

### Dataset ID: 1

In [13]:
!python ../../pipelines/create_D_from_games.py 1 1

team2           Alabama  Alcorn_St  Arizona  ...  Wisconsin  Wyoming  Xavier
team1                                        ...                            
Alabama             NaN        NaN      NaN  ...        NaN      NaN     NaN
Alcorn_St           NaN        NaN      NaN  ...        NaN      NaN     NaN
Arizona             NaN        NaN      NaN  ...        NaN      NaN     NaN
Boston_College      NaN        NaN      NaN  ...        NaN      NaN     NaN
Boston_Univ         NaN        NaN      NaN  ...        NaN      NaN     NaN
...                 ...        ...      ...  ...        ...      ...     ...
Wake_Forest         NaN        NaN      0.0  ...        NaN      NaN     NaN
Winthrop            NaN        NaN      NaN  ...        NaN      NaN     0.0
Wisconsin           NaN        NaN      NaN  ...        NaN      NaN     NaN
Wyoming             NaN        NaN      NaN  ...        NaN      NaN     NaN
Xavier              NaN        NaN      NaN  ...        NaN      NaN     NaN

## LOP Card Creation

### Dataset ID: 1